In [217]:
import pandas as pd
import numpy as np

In [218]:
df=pd.read_csv('./raw_data/t-2_raw_외감.csv',index_col=False,encoding='euc-kr')

In [219]:
df.isna().sum()

회사명                   0
거래소코드                 0
회계년도                  0
산업분류코드                0
부채비율                 40
당좌비율                 40
유동비율                 40
이자보상배율               40
OCF(천원)              38
이자비용(천원)             38
차입금의존도               40
자기자본구성비율             40
당기순이익(천원)            38
감가상각비(백만원)         1100
금융비용(백만원)           151
단기차입금(천원)            38
유동성장기부채(천원)          38
장기차입금(천원)            38
법인세비용차감전손익(백만원)      49
매출액영업이익률             39
자기자본순이익률             39
총자본순이익률              39
매출액(천원)              38
총자본회전률               40
자기자본회전률              40
운전자본회전률              40
순운전자본회전률             40
재고자산회전률              40
당좌자산회전률              40
유동자산회전률              40
매출액증가율               39
총자본증가율               39
자기자본증가율              39
순이익증가율               39
유형자산증가율              39
유동자산증가율              39
재고자산증가율              39
영업이익증가율              39
총자본투자효율              42
부가가치율                42
노동소득분배율              42
자본분배율           

In [220]:
df.dropna(subset='자산총계(천원)', axis=0, inplace=True)
df.dropna(subset='법인세비용차감전손익(백만원)', axis=0, inplace=True)
df.dropna(subset='부가가치율', axis=0, inplace=True)
df.dropna(subset='부채비율', axis=0, inplace=True)
df.dropna(subset='매출액영업이익률', axis=0, inplace=True)

In [221]:
df['감가상각비(백만원)'] = df['감가상각비(백만원)'].fillna(0)
df['금융비용(백만원)'] = df['금융비용(백만원)'].fillna(0)

In [222]:
import pandas as pd
import numpy as np

def trim_outliers(series):
    # 양쪽 극단치를 제외한 범위 계산
    lower_percentile = series.quantile(0.01)
    upper_percentile = series.quantile(0.99)

    # 극단치를 제외한 데이터 선택
    trimmed_series = series[(series >= lower_percentile) & (series <= upper_percentile)]

    return trimmed_series

In [223]:
a=df[~(df['이자비용(천원)']==0)]['이자보상배율']
df1 = trim_outliers(a)
df1.agg(['max','min'])

max    8305.17
min    -168.57
Name: 이자보상배율, dtype: float64

In [224]:
df['이자보상배율'] = (df['영업손익(천원)'] + df['감가상각비(백만원)']) / df['이자비용(천원)']

In [225]:
df['이자보상배율'].replace(np.inf,8305.17,inplace=True)
df['이자보상배율'].replace(-(np.inf),-168.57,inplace=True)

---
- 파생변수 만들기

In [226]:
df['OCF이자보상배율'] = df['OCF(천원)']/df['이자비용(천원)']   ## 파보배

df['부채상환계수'] = (df['당기순이익(천원)']+(df['감가상각비(백만원)']*1000)+(df['금융비용(백만원)']*1000))/(df['단기차입금(천원)']+df['유동성장기부채(천원)']+(df['금융비용(백만원)']*1000))
df['장기부채상환능력'] = df['장기차입금(천원)']/((df['법인세비용차감전손익(백만원)']*1000)+(df['감가상각비(백만원)']*1000)+df['이자비용(천원)'])
df['매출액대비금융비용상환능력'] = df['매출액(천원)']/(df['금융비용(백만원)']*1000)  

df['전체현금흐름(천원)'] = df['OCF(천원)']+df['ICF(천원)']+df['FCF(천원)']
df['잉여현금흐름(FCF)']= df['OCF(천원)']+df['ICF(천원)']

df['연구개발비대비매출액'] = df['연구개발비(천원)']/df['매출액(천원)']
df['매출액대비현금흐름'] = df['전체현금흐름(천원)'] / df['매출액(천원)']
df['매출액대비잉여현금흐름'] = df['잉여현금흐름(FCF)']/ df['매출액(천원)']

df['총자산대비현금흐름'] = df['전체현금흐름(천원)'] / df['자산총계(천원)']
df['총자산대비영업현금흐름'] = df['OCF(천원)'] / df['자산총계(천원)']
df['총자산대비잉여현금흐름'] = df['잉여현금흐름(FCF)']/df['자산총계(천원)']

# 잉여현금흐름(FCF)

In [227]:
for i in df.index :
    if (df.loc[i,'OCF(천원)'] < 0) & (df.loc[i,'ICF(천원)'] < 0) &(df.loc[i,'FCF(천원)'] > 0) :
        df.loc[i,'기업수명주기'] = '도입기'
    elif (df.loc[i,'OCF(천원)'] > 0) & (df.loc[i,'ICF(천원)'] < 0) &(df.loc[i,'FCF(천원)'] > 0) :
        df.loc[i,'기업수명주기'] = '성장기'
    elif (df.loc[i,'OCF(천원)'] > 0) & (df.loc[i,'ICF(천원)'] < 0) &(df.loc[i,'FCF(천원)'] < 0) :
        df.loc[i,'기업수명주기'] = '성숙기'
    elif (df.loc[i,'OCF(천원)'] < 0) & (df.loc[i,'ICF(천원)'] > 0):
        df.loc[i,'기업수명주기'] = '쇠퇴기'
    else :
        df.loc[i,'기업수명주기'] = '수축기'

- OCF이자보상배율 구하기
    - 결측치를 0으로 채우기
    - inf : 절삭하고 최대값 대입
    - -inf : 절삭하고 최소값 대입

In [228]:
a=df[~(df['이자비용(천원)']==0)]['OCF이자보상배율']
df1 = trim_outliers(a)
df1.agg(['max','min'])

max    8880.582955
min    -529.718651
Name: OCF이자보상배율, dtype: float64

In [229]:
df['OCF이자보상배율'].replace(np.inf,8880.582955,inplace=True)
df['OCF이자보상배율'].replace(-(np.inf),-529.718651,inplace=True)

In [230]:
df=df[~(df['매출액(천원)']==0.0)]
df=df[~(df['매출액(천원)']<0.0)]

In [231]:
df=df[~(df['자산총계(천원)']==0)]
df=df[~(df['자산총계(천원)']<0)]

In [232]:
# pd.set_option('display.max_seq_item',None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
df.isna().sum()

회사명                 0
거래소코드               0
회계년도                0
산업분류코드              0
부채비율                0
당좌비율                0
유동비율                0
이자보상배율              7
OCF(천원)             0
이자비용(천원)            0
차입금의존도              0
자기자본구성비율            0
당기순이익(천원)           0
감가상각비(백만원)          0
금융비용(백만원)           0
단기차입금(천원)           0
유동성장기부채(천원)         0
장기차입금(천원)           0
법인세비용차감전손익(백만원)     0
매출액영업이익률            0
자기자본순이익률            0
총자본순이익률             0
매출액(천원)             0
총자본회전률              0
자기자본회전률             0
운전자본회전률             0
순운전자본회전률            0
재고자산회전률             0
당좌자산회전률             0
유동자산회전률             0
매출액증가율              0
총자본증가율              0
자기자본증가율             0
순이익증가율              0
유형자산증가율             0
유동자산증가율             0
재고자산증가율             0
영업이익증가율             0
총자본투자효율             0
부가가치율               0
노동소득분배율             0
자본분배율               0
이윤분배율               0
ICF(천원)             0
FCF(천원)             0
자산총계(천원)  

In [233]:
len(df[(df['이자비용(천원)']==0) &(df['OCF(천원)']==0)])

33

In [234]:
df['OCF이자보상배율'].fillna(0,inplace=True)

In [235]:
# len(df[(df['당기순이익(천원)']==0) &(df['감가상각비(백만원)']==0) &(df['금융비용(백만원)']==0)])
len(df[(df['단기차입금(천원)']==0) &(df['유동성장기부채(천원)']==0) &(df['금융비용(백만원)']==0) & (df['당기순이익(천원)']==0) &(df['감가상각비(백만원)']==0)])

3

In [236]:
df['부채상환계수'].fillna(0,inplace=True)
df['장기부채상환능력'].fillna(0,inplace=True)

In [237]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
df.isna().sum()

회사명                0
거래소코드              0
회계년도               0
산업분류코드             0
부채비율               0
당좌비율               0
유동비율               0
이자보상배율             7
OCF(천원)            0
이자비용(천원)           0
차입금의존도             0
자기자본구성비율           0
당기순이익(천원)          0
감가상각비(백만원)         0
금융비용(백만원)          0
단기차입금(천원)          0
유동성장기부채(천원)        0
장기차입금(천원)          0
법인세비용차감전손익(백만원)    0
매출액영업이익률           0
자기자본순이익률           0
총자본순이익률            0
매출액(천원)            0
총자본회전률             0
자기자본회전률            0
운전자본회전률            0
순운전자본회전률           0
재고자산회전률            0
당좌자산회전률            0
유동자산회전률            0
매출액증가율             0
총자본증가율             0
자기자본증가율            0
순이익증가율             0
유형자산증가율            0
유동자산증가율            0
재고자산증가율            0
영업이익증가율            0
총자본투자효율            0
부가가치율              0
노동소득분배율            0
자본분배율              0
이윤분배율              0
ICF(천원)            0
FCF(천원)            0
자산총계(천원)           0
부채총계(천원)           0
자본총계(천원)     

- 결측치 확인 끝!!

---
- inf, -inf 확인하기

In [238]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
df.isin([np.inf,-np.inf]).sum()

회사명                  0
거래소코드                0
회계년도                 0
산업분류코드               0
부채비율                 0
당좌비율                 0
유동비율                 0
이자보상배율               0
OCF(천원)              0
이자비용(천원)             0
차입금의존도               0
자기자본구성비율             0
당기순이익(천원)            0
감가상각비(백만원)           0
금융비용(백만원)            0
단기차입금(천원)            0
유동성장기부채(천원)          0
장기차입금(천원)            0
법인세비용차감전손익(백만원)      0
매출액영업이익률             0
자기자본순이익률             0
총자본순이익률              0
매출액(천원)              0
총자본회전률               0
자기자본회전률              0
운전자본회전률              0
순운전자본회전률             0
재고자산회전률              0
당좌자산회전률              0
유동자산회전률              0
매출액증가율               0
총자본증가율               0
자기자본증가율              0
순이익증가율               0
유형자산증가율              0
유동자산증가율              0
재고자산증가율              0
영업이익증가율              0
총자본투자효율              0
부가가치율                0
노동소득분배율              0
자본분배율                0
이윤분배율                0
ICF(천원)    

In [239]:
len(df[(df['단기차입금(천원)']==0) &(df['유동성장기부채(천원)']==0) &(df['금융비용(백만원)']==0)])

157

In [240]:
len(df[(df['법인세비용차감전손익(백만원)']==0) &(df['감가상각비(백만원)']==0) &(df['이자비용(천원)']==0)])

4

In [241]:
len(df[df['금융비용(백만원)']==0])

327

In [242]:
df['부채상환계수'] = df['부채상환계수'].replace([np.inf, -np.inf], 0)
df['장기부채상환능력'] = df['장기부채상환능력'].replace([np.inf, -np.inf], 0)
df['매출액대비금융비용상환능력'] = df['매출액대비금융비용상환능력'].replace([np.inf, -np.inf], 0)

In [243]:
pd.reset_option('display.max_rows', None)
pd.reset_option('display.max_columns', None)
pd.reset_option('display.width', None)
df.isin([np.inf,-np.inf]).sum()

회사명            0
거래소코드          0
회계년도           0
산업분류코드         0
부채비율           0
              ..
매출액대비잉여현금흐름    0
총자산대비현금흐름      0
총자산대비영업현금흐름    0
총자산대비잉여현금흐름    0
기업수명주기         0
Length: 68, dtype: int64

---
- 데이터의 결측치, 무한대값 모두 0개

In [244]:
df['t-2감사의견코드'].value_counts()

0.0    135563
1.0      2058
Name: t-2감사의견코드, dtype: int64

In [245]:
order =['회사명', '거래소코드', '회계년도', 'Year', '산업분류코드', '산업분류','부채비율', '당좌비율', '유동비율', '이자보상배율',
       'OCF(천원)', '이자비용(천원)', '차입금의존도', '자기자본구성비율', '당기순이익(천원)', '감가상각비(백만원)',
       '금융비용(백만원)', '단기차입금(천원)', '유동성장기부채(천원)', '장기차입금(천원)', '법인세비용차감전손익(백만원)',
       '매출액영업이익률', '자기자본순이익률', '총자본순이익률', '매출액(천원)', '총자본회전률', '자기자본회전률',
       '운전자본회전률', '순운전자본회전률', '재고자산회전률', '당좌자산회전률', '유동자산회전률', '매출액증가율',
       '총자본증가율', '자기자본증가율', '순이익증가율', '유형자산증가율', '유동자산증가율', '재고자산증가율',
       '영업이익증가율', '총자본투자효율', '부가가치율', '노동소득분배율', '자본분배율', '이윤분배율', 'ICF(천원)',
       'FCF(천원)', '자산총계(천원)', '부채총계(천원)', '자본총계(천원)',  '자본금(천원)',
       '영업손익(천원)', '연구개발비(천원)', '자본잠식률', 'OCF이자보상배율',
       '부채상환계수', '장기부채상환능력', '매출액대비금융비용상환능력', '전체현금흐름(천원)', '잉여현금흐름(FCF)',
       '연구개발비대비매출액', '매출액대비현금흐름', '매출액대비잉여현금흐름', '총자산대비현금흐름', '총자산대비영업현금흐름',
       '총자산대비잉여현금흐름', '기업수명주기', '평균부채비율초과여부', 't-2감사의견코드']

In [246]:
df = df.reindex(columns=order)

---

- train/test split (2.3/7.7)

In [247]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test= train_test_split(df.drop('t-2감사의견코드',axis=1),df['t-2감사의견코드'],test_size=0.23,stratify=df['t-2감사의견코드'],random_state=0) 

In [248]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))


105968
31653
105968
31653


In [249]:
train_data = pd.concat([X_train,y_train],axis=1)

In [250]:
test_data= pd.concat([X_test,y_test],axis=1 )

In [251]:
train_data['자산총계(천원)'] = train_data['자산총계(천원)'] * 1000
train_data['log자산총계'] = np.log(train_data['자산총계(천원)'])

test_data['자산총계(천원)'] = test_data['자산총계(천원)'] * 1000
test_data['log자산총계'] = np.log(test_data['자산총계(천원)'])

In [252]:
train_data=train_data[['회사명', '거래소코드', 'Year', '부채비율', '당좌비율',
       '유동비율', '이자보상배율', '차입금의존도', '자기자본구성비율',
       '매출액영업이익률', '자기자본순이익률', '총자본순이익률', '총자본회전률', '자기자본회전률', '운전자본회전률', '순운전자본회전률', '재고자산회전률',
       '당좌자산회전률', '유동자산회전률', '매출액증가율', '총자본증가율', '자기자본증가율', '순이익증가율',
       '유형자산증가율', '유동자산증가율', '재고자산증가율', '영업이익증가율', '총자본투자효율', '부가가치율',
       '노동소득분배율', '자본분배율', '이윤분배율', 'OCF이자보상배율', '부채상환계수', '장기부채상환능력', '매출액대비금융비용상환능력',
       '연구개발비대비매출액', '매출액대비현금흐름', '매출액대비잉여현금흐름', '총자산대비현금흐름',
       '총자산대비영업현금흐름', '총자산대비잉여현금흐름', '기업수명주기', 'log자산총계',
       't-2감사의견코드']]
test_data= test_data[['회사명', '거래소코드', 'Year', '부채비율', '당좌비율',
       '유동비율', '이자보상배율', '차입금의존도', '자기자본구성비율',
       '매출액영업이익률', '자기자본순이익률', '총자본순이익률', '총자본회전률', '자기자본회전률', '운전자본회전률', '순운전자본회전률', '재고자산회전률',
       '당좌자산회전률', '유동자산회전률', '매출액증가율', '총자본증가율', '자기자본증가율', '순이익증가율',
       '유형자산증가율', '유동자산증가율', '재고자산증가율', '영업이익증가율', '총자본투자효율', '부가가치율',
       '노동소득분배율', '자본분배율', '이윤분배율', 'OCF이자보상배율', '부채상환계수', '장기부채상환능력', '매출액대비금융비용상환능력',
       '연구개발비대비매출액', '매출액대비현금흐름', '매출액대비잉여현금흐름', '총자산대비현금흐름',
       '총자산대비영업현금흐름', '총자산대비잉여현금흐름', '기업수명주기', 'log자산총계',
       't-2감사의견코드']]

In [258]:
train_data=train_data[['부채비율', '당좌비율',
       '유동비율', '이자보상배율', '차입금의존도', '자기자본구성비율',
       '매출액영업이익률', '자기자본순이익률', '총자본순이익률', '총자본회전률', '자기자본회전률', '운전자본회전률', '순운전자본회전률', '재고자산회전률',
       '당좌자산회전률', '유동자산회전률', '매출액증가율', '총자본증가율', '자기자본증가율', '순이익증가율',
       '유형자산증가율', '유동자산증가율', '재고자산증가율', '영업이익증가율', '총자본투자효율', '부가가치율',
       '노동소득분배율', '자본분배율', '이윤분배율', 'OCF이자보상배율', '부채상환계수', '장기부채상환능력', '매출액대비금융비용상환능력',
       '연구개발비대비매출액', '매출액대비현금흐름', '매출액대비잉여현금흐름', '총자산대비현금흐름',
       '총자산대비영업현금흐름', '총자산대비잉여현금흐름',  'log자산총계',
       't-2감사의견코드']]
test_data= test_data[[ '부채비율', '당좌비율',
       '유동비율', '이자보상배율', '차입금의존도', '자기자본구성비율',
       '매출액영업이익률', '자기자본순이익률', '총자본순이익률', '총자본회전률', '자기자본회전률', '운전자본회전률', '순운전자본회전률', '재고자산회전률',
       '당좌자산회전률', '유동자산회전률', '매출액증가율', '총자본증가율', '자기자본증가율', '순이익증가율',
       '유형자산증가율', '유동자산증가율', '재고자산증가율', '영업이익증가율', '총자본투자효율', '부가가치율',
       '노동소득분배율', '자본분배율', '이윤분배율', 'OCF이자보상배율', '부채상환계수', '장기부채상환능력', '매출액대비금융비용상환능력',
       '연구개발비대비매출액', '매출액대비현금흐름', '매출액대비잉여현금흐름', '총자산대비현금흐름',
       '총자산대비영업현금흐름', '총자산대비잉여현금흐름',  'log자산총계',
       't-2감사의견코드']]

In [264]:
feature = train_data
target = test_data

In [262]:
feature.dropna(inplace=True)
target.dropna(inplace=True)

C:\Users\dgh06\AppData\Local\Temp\ipykernel_15880\1395767223.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature.dropna(inplace=True)
C:\Users\dgh06\AppData\Local\Temp\ipykernel_15880\1395767223.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target.dropna(inplace=True)


In [266]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
import statsmodels.api as sm
import numpy as np
lr_clf = LogisticRegression()

feature = train_data.drop('t-2감사의견코드',axis=1)
target = train_data[['t-2감사의견코드']]

logit = SelectFromModel(LogisticRegression())
logit.fit(feature, target)
logit_support = logit.get_support()
lr_feature = feature.loc[:,logit_support].columns.tolist()

c:\Users\dgh06\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\dgh06\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [267]:
lr_feature


['당좌비율',
 '유동비율',
 '차입금의존도',
 '자기자본구성비율',
 '재고자산회전률',
 '자기자본증가율',
 '재고자산증가율',
 '영업이익증가율',
 '총자본투자효율',
 '노동소득분배율',
 '자본분배율',
 'log자산총계']

--- 
- Feature selection

In [268]:
X_train_int=train_data[['당좌비율',
 '유동비율',
 '차입금의존도',
 '자기자본구성비율',
 '재고자산회전률',
 '자기자본증가율',
 '재고자산증가율',
 '영업이익증가율',
 '총자본투자효율',
 '노동소득분배율',
 '자본분배율',
 'log자산총계']]

X_test_int=test_data[['당좌비율',
 '유동비율',
 '차입금의존도',
 '자기자본구성비율',
 '재고자산회전률',
 '자기자본증가율',
 '재고자산증가율',
 '영업이익증가율',
 '총자본투자효율',
 '노동소득분배율',
 '자본분배율',
 'log자산총계']]


In [269]:
X_train = train_data.drop('t-2감사의견코드',axis=1)
y_train = train_data[['t-2감사의견코드']]

X_test = test_data.drop('t-2감사의견코드',axis=1)
y_test = test_data[['t-2감사의견코드']]

In [270]:
from imblearn.over_sampling import RandomOverSampler

# RandomOverSampler 객체 생성
ros = RandomOverSampler()

# Random Over Sampling 적용
X_resampled, y_resampled = ros.fit_resample(X_train_int, y_train)


In [276]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate_random_forest(X_train, y_train, X_test, y_test):
    # Stratified 5-fold 교차검증 설정
    cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=0)

    # 최적 하이퍼파라미터 설정 (여기서는 고정값으로 사용)
    best_params = {'max_depth': 12, 'min_samples_leaf': 20, 'max_features':'auto', 'n_estimators':200}

    # 각 fold 별 평가 지표를 저장할 리스트 초기화
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []
    confusion_matrix_list = []

    best_f1_score = 0
    best_model = None

    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train), 1):
        X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_test_fold, y_test_fold = X_train.iloc[test_idx], y_train.iloc[test_idx]

        # 랜덤 포레스트 모델 초기화
        model = RandomForestClassifier(max_depth=best_params['max_depth'],
                                       min_samples_leaf=best_params['min_samples_leaf'],
                                       n_estimators=best_params['n_estimators'],
                                       max_features= best_params['max_features'],
                                       random_state=0)

        # 모델 학습
        model.fit(X_train_fold, y_train_fold)

        # 테스트 데이터에 대한 예측 확률 얻기
        probabilities = model.predict_proba(X_test_fold)

        # threshold를 0.4로 설정하여 예측 클래스를 조정
        threshold = 0.1
        predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)

        # 평가 지표 계산
        accuracy = accuracy_score(y_test_fold, predicted_classes)
        precision = precision_score(y_test_fold, predicted_classes)
        recall = recall_score(y_test_fold, predicted_classes)
        f1 = f1_score(y_test_fold, predicted_classes)
        conf_matrix = confusion_matrix(y_test_fold, predicted_classes)

        # 각 fold 별 평가 지표를 리스트에 추가
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_score_list.append(f1)
        confusion_matrix_list.append(conf_matrix)

        print(f"Fold {fold_idx}")
        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 score: {f1}")
        print("Confusion Matrix:")
        print(conf_matrix)
        print("------------------------------")

        # 가장 좋은 f1-score 값을 가진 모델을 저장
        if f1 > best_f1_score:
            best_f1_score = f1
            best_model = model

    # 가장 좋은 f1-score 값을 가진 모델로 최종 예측 수행
    probabilities_final = best_model.predict_proba(X_test)
    y_pred_final = (probabilities_final[:, 1] > threshold).astype(np.int)

    # 평가 지표 계산
    accuracy_final = accuracy_score(y_test, y_pred_final)
    precision_final = precision_score(y_test, y_pred_final)
    recall_final = recall_score(y_test, y_pred_final)
    f1_final = f1_score(y_test, y_pred_final)
    conf_matrix_final = confusion_matrix(y_test, y_pred_final)

    print("Final Test Results")
    print(f"Accuracy: {accuracy_final}")
    print(f"Precision: {precision_final}")
    print(f"Recall: {recall_final}")
    print(f"F1 score: {f1_final}")
    print("Confusion Matrix:")
    print(conf_matrix_final)

    return accuracy_list, precision_list, recall_list, f1_score_list


In [277]:
evaluate_random_forest(X_train_int, y_train, X_test_int, y_test)

C:\Users\dgh06\AppData\Local\Temp\ipykernel_15880\2601399893.py:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_fold, y_train_fold)
c:\Users\dgh06\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\dgh06\AppData\Local\Temp\ipykernel_15880\2601399893.py:41: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish 

Fold 1
Accuracy: 0.9803895662677891
Precision: 0.09539473684210527
Recall: 0.03656998738965952
F1 score: 0.0528714676390155
Confusion Matrix:
[[51914   275]
 [  764    29]]
------------------------------


C:\Users\dgh06\AppData\Local\Temp\ipykernel_15880\2601399893.py:41: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)


Fold 2
Accuracy: 0.981125309073064
Precision: 0.09375
Recall: 0.030303030303030304
F1 score: 0.04580152671755725
Confusion Matrix:
[[51957   232]
 [  768    24]]
------------------------------
Final Test Results
Accuracy: 0.9802533885185302
Precision: 0.05813953488372093
Recall: 0.021141649048625793
F1 score: 0.0310077519379845
Confusion Matrix:
[[31016   162]
 [  463    10]]


C:\Users\dgh06\AppData\Local\Temp\ipykernel_15880\2601399893.py:73: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_pred_final = (probabilities_final[:, 1] > threshold).astype(np.int)


([0.9803895662677891, 0.981125309073064],
 [0.09539473684210527, 0.09375],
 [0.03656998738965952, 0.030303030303030304],
 [0.0528714676390155, 0.04580152671755725])